In [75]:
import pandas as pd
df = pd.read_csv('tennis_final.csv')

In [76]:
print(df.shape)
df.head()

(168302, 56)


,Unnamed: 0,Unnamed: 0.1,Player A,Player A 1stIn,Player A 1stWon,Player A 2ndwon,Player A Entry,Player A Hand,Player A Height,Player A ID,Player A Name,Player A SvGms,Player A ace,Player A bpFaced,Player A bpSaved,Player A df,Player A ioc,Player A points,Player A rank,Player A seed,Player A svpt,Player B,Player B 1stIn,Player B 1stWon,Player B 2ndwon,Player B Entry,Player B Hand,Player B Height,Player B ID,Player B Name,Player B SvGms,Player B ace,Player B bpFaced,Player B bpSaved,Player B df,Player B ioc,Player B points,Player B rank,Player B seed,Player B svpt,Winner,best_of,draw_size,match_num,minutes,players,round,score,surface,tourney_date,tourney_id,tourney_level,tourney_name,winner_name,year,Winner2
0,64970,915,Sebastien Grosjean,48.0,32.0,25.0,NaN,R,175.0,103206,27.81,16.0,6.0,18.0,11.0,2.0,FRA,1450.0,20.0,19.0,106.0,David Ferrer,80.0,50.0,16.0,NaN,R,175.0,103970,23.96,16.0,1.0,11.0,5.0,5.0,ESP,1700.0,11.0,11.0,116.0,David Ferrer,3,NaN,71,168.0,Player 2,R32,4-6 7-5 6-4,Hard,20060320,2006-403,M,Miami Masters,David Ferrer,2006,Player B
1,89521,1453,Jaime Fillol,NaN,NaN,NaN,NaN,R,180.0,100115,26.95,NaN,NaN,NaN,NaN,NaN,CHI,NaN,NaN,NaN,NaN,Robert Lutz,NaN,NaN,NaN,NaN,R,180.0,100144,25.71,NaN,NaN,NaN,NaN,NaN,USA,NaN,NaN,14.0,NaN,Jaime Fillol,3,NaN,15,NaN,Player 1,R32,7-6 4-6 6-2,Hard,19730514,1973-413,A,Las Vegas,Jaime Fillol,1973,Player A
2,72519,317,Frederico Gil,45.0,31.0,18.0,NaN,R,178.0,104523,25.86,13.0,5.0,8.0,4.0,3.0,POR,606.0,80.0,NaN,81.0,Tommy Robredo,49.0,32.0,14.0,NaN,R,180.0,103990,28.75,14.0,5.0,9.0,5.0,4.0,ESP,1060.0,40.0,6.0,73.0,Tommy Robredo,3,NaN,13,112.0,Player 2,R32,2-6 6-3 6-4,Clay,20110131,2011-505,A,Santiago,Tommy Robredo,2011,Player B
3,114370,2112,Tim Wilkison,NaN,NaN,NaN,NaN,L,180.0,100631,26.72,NaN,NaN,NaN,NaN,NaN,USA,NaN,28.0,13.0,NaN,Vijay Amritraj,NaN,NaN,NaN,NaN,R,193.0,100321,32.66,NaN,NaN,NaN,NaN,NaN,IND,NaN,84.0,NaN,NaN,Tim Wilkison,3,NaN,30,NaN,Player 1,R32,6-7 2-1 RET,Hard,19860811,1986-421,M,Montreal / Toronto,Tim Wilkison,1986,Player A
4,40996,2575,Derrick Rostagno,51.0,32.0,8.0,NaN,R,185.0,101196,26.77,9.0,4.0,4.0,2.0,2.0,USA,1057.0,27.0,5.0,73.0,Richey Reneberg,35.0,29.0,14.0,NaN,R,180.0,101191,26.83,10.0,1.0,0.0,0.0,0.0,USA,520.0,69.0,NaN,52.0,Richey Reneberg,3,NaN,21,75.0,Player 2,R16,6-1 7-6(2),Hard,19920803,1992-423,A,Los Angeles,Richey Reneberg,1992,Player B


In [77]:
df.loc[(df['Player B']) == (df['Player A'])]

,Unnamed: 0,Unnamed: 0.1,Player A,Player A 1stIn,Player A 1stWon,Player A 2ndwon,Player A Entry,Player A Hand,Player A Height,Player A ID,Player A Name,Player A SvGms,Player A ace,Player A bpFaced,Player A bpSaved,Player A df,Player A ioc,Player A points,Player A rank,Player A seed,Player A svpt,Player B,Player B 1stIn,Player B 1stWon,Player B 2ndwon,Player B Entry,Player B Hand,Player B Height,Player B ID,Player B Name,Player B SvGms,Player B ace,Player B bpFaced,Player B bpSaved,Player B df,Player B ioc,Player B points,Player B rank,Player B seed,Player B svpt,Winner,best_of,draw_size,match_num,minutes,players,round,score,surface,tourney_date,tourney_id,tourney_level,tourney_name,winner_name,year,Winner2
549,67,139,U Unknown,NaN,NaN,NaN,NaN,U,NaN,199999,NaN,NaN,NaN,NaN,NaN,NaN,UNK,NaN,NaN,NaN,NaN,U Unknown,NaN,NaN,NaN,NaN,U,NaN,199999,NaN,NaN,NaN,NaN,NaN,NaN,UNK,NaN,NaN,NaN,NaN,U Unknown,5,64.0,5,NaN,Player 2,R64,6-2 5-7 6-4 6-3,Clay,19680607,1968-425,A,Barcelona,U Unknown,1968,Player A
88041,7012,1662,U Unknown,NaN,NaN,NaN,NaN,U,NaN,199999,NaN,NaN,NaN,NaN,NaN,NaN,UNK,NaN,NaN,NaN,NaN,U Unknown,NaN,NaN,NaN,NaN,U,NaN,199999,NaN,NaN,NaN,NaN,NaN,NaN,UNK,NaN,NaN,NaN,NaN,U Unknown,3,NaN,278,NaN,Player 2,R32,4-6 6-2 6-2,Grass,19740601,1974-720,A,Surbiton,U Unknown,1974,Player A


In [78]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

X = df.drop(columns='Winner2')
y = df['Winner2']

X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=20000, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=20000, 
    random_state=42)

print('X_train shape', X_train.shape)
print('y_train shape', y_train.shape)
print('X_val shape', X_val.shape)
print('y_val shape', y_val.shape)
print('X_test shape', X_test.shape)
print('y_test shape', y_test.shape)

X_train shape (128302, 55)
y_train shape (128302,)
X_val shape (20000, 55)
y_val shape (20000,)
X_test shape (20000, 55)
y_test shape (20000,)


In [79]:
pd.set_option('display.max_columns', 500)
df.head()

,Unnamed: 0,Unnamed: 0.1,Player A,Player A 1stIn,Player A 1stWon,Player A 2ndwon,Player A Entry,Player A Hand,Player A Height,Player A ID,Player A Name,Player A SvGms,Player A ace,Player A bpFaced,Player A bpSaved,Player A df,Player A ioc,Player A points,Player A rank,Player A seed,Player A svpt,Player B,Player B 1stIn,Player B 1stWon,Player B 2ndwon,Player B Entry,Player B Hand,Player B Height,Player B ID,Player B Name,Player B SvGms,Player B ace,Player B bpFaced,Player B bpSaved,Player B df,Player B ioc,Player B points,Player B rank,Player B seed,Player B svpt,Winner,best_of,draw_size,match_num,minutes,players,round,score,surface,tourney_date,tourney_id,tourney_level,tourney_name,winner_name,year,Winner2
0,64970,915,Sebastien Grosjean,48.0,32.0,25.0,NaN,R,175.0,103206,27.81,16.0,6.0,18.0,11.0,2.0,FRA,1450.0,20.0,19.0,106.0,David Ferrer,80.0,50.0,16.0,NaN,R,175.0,103970,23.96,16.0,1.0,11.0,5.0,5.0,ESP,1700.0,11.0,11.0,116.0,David Ferrer,3,NaN,71,168.0,Player 2,R32,4-6 7-5 6-4,Hard,20060320,2006-403,M,Miami Masters,David Ferrer,2006,Player B
1,89521,1453,Jaime Fillol,NaN,NaN,NaN,NaN,R,180.0,100115,26.95,NaN,NaN,NaN,NaN,NaN,CHI,NaN,NaN,NaN,NaN,Robert Lutz,NaN,NaN,NaN,NaN,R,180.0,100144,25.71,NaN,NaN,NaN,NaN,NaN,USA,NaN,NaN,14.0,NaN,Jaime Fillol,3,NaN,15,NaN,Player 1,R32,7-6 4-6 6-2,Hard,19730514,1973-413,A,Las Vegas,Jaime Fillol,1973,Player A
2,72519,317,Frederico Gil,45.0,31.0,18.0,NaN,R,178.0,104523,25.86,13.0,5.0,8.0,4.0,3.0,POR,606.0,80.0,NaN,81.0,Tommy Robredo,49.0,32.0,14.0,NaN,R,180.0,103990,28.75,14.0,5.0,9.0,5.0,4.0,ESP,1060.0,40.0,6.0,73.0,Tommy Robredo,3,NaN,13,112.0,Player 2,R32,2-6 6-3 6-4,Clay,20110131,2011-505,A,Santiago,Tommy Robredo,2011,Player B
3,114370,2112,Tim Wilkison,NaN,NaN,NaN,NaN,L,180.0,100631,26.72,NaN,NaN,NaN,NaN,NaN,USA,NaN,28.0,13.0,NaN,Vijay Amritraj,NaN,NaN,NaN,NaN,R,193.0,100321,32.66,NaN,NaN,NaN,NaN,NaN,IND,NaN,84.0,NaN,NaN,Tim Wilkison,3,NaN,30,NaN,Player 1,R32,6-7 2-1 RET,Hard,19860811,1986-421,M,Montreal / Toronto,Tim Wilkison,1986,Player A
4,40996,2575,Derrick Rostagno,51.0,32.0,8.0,NaN,R,185.0,101196,26.77,9.0,4.0,4.0,2.0,2.0,USA,1057.0,27.0,5.0,73.0,Richey Reneberg,35.0,29.0,14.0,NaN,R,180.0,101191,26.83,10.0,1.0,0.0,0.0,0.0,USA,520.0,69.0,NaN,52.0,Richey Reneberg,3,NaN,21,75.0,Player 2,R16,6-1 7-6(2),Hard,19920803,1992-423,A,Los Angeles,Richey Reneberg,1992,Player B


In [80]:
def wrangle(X):
    X = X.copy()
    
    #Drop the leaking columns.
    X = X.drop(columns=['winner_name', 'Unnamed: 0', 'Unnamed: 0.1', 'winner_name', 'players', 'tourney_name', 
                       'score', 'minutes', 'tourney_date', 'Winner'])
    
    
  
    
    X = X.rename(columns={'Player A': 'Player_A', 'Player B':'Player_B'})
    return X

X_train = wrangle(X_train)
X_val   = wrangle(X_val)
X_test  = wrangle(X_test)

print('X_train shape', X_train.shape)
print('X_val shape', X_val.shape)
print('X_test shape', X_test.shape)

X_train shape (128302, 46)
X_val shape (20000, 46)
X_test shape (20000, 46)


In [81]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
import category_encoders as ce

pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='median'),
    XGBClassifier(n_estimators=100, n_jobs=-1)
)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['Player_A', 'Player A Entry',
                                      'Player A Hand', 'Player A ioc',
                                      'Player_B', 'Player B Entry',
                                      'Player B Hand', 'Player B ioc', 'round',
                                      'surface', 'tourney_id',
                                      'tourney_level'],
                                drop_invariant=False, handle_missing='value',
                                handle_unknown='value',
                                mapping=[{'col': 'Player_A',
                                          'data_type': dtype('O'),
                                          'mapping': Cr...
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,


In [61]:
proecessed = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='median'),
)

X_train_processed = proecessed.fit_transform(X_train)
X_val_processed = proecessed.transform(X_val)
X_test_processed = proecessed.transform(X_test)

In [31]:
from joblib import dump
dump(model, 'pipeline.joblib', compress=True)

['pipeline.joblib']

In [82]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_train_processed)

print('Training Accuracy', accuracy_score(y_train, y_pred))

Training Accuracy 0.7919518012189989


In [83]:
import joblib
import sklearn
import category_encoders as ce
import xgboost
print(f'xgboost=={xgboost.__version__}')
print(f'joblib=={joblib.__version__}')
print(f'scikit-learn=={sklearn.__version__}')
print(f'category_encoders=={ce.__version__}')

xgboost==0.90
joblib==0.14.0
scikit-learn==0.21.3
category_encoders==2.1.0


In [84]:
y_pred = model.predict(X_val_processed)

print('Validation Accuracy', accuracy_score(y_val, y_pred))

Validation Accuracy 0.7872


In [85]:
y_pred = model.predict(X_test_processed)

print('test Accuracy', accuracy_score(y_test, y_pred))

test Accuracy 0.78005


In [35]:
y_train.value_counts(normalize=True)     #This is our baseline, that Player B wins slightly over 50% of the time.

Player B    0.50233
Player A    0.49767
Name: Winner2, dtype: float64

In [36]:
import eli5
from eli5.sklearn import PermutationImportance

permuter = PermutationImportance(
    model, 
    scoring='accuracy', 
    n_iter=2, 
    random_state=42
)

permuter.fit(X_val_processed, y_val)

PermutationImportance(cv='prefit',
                      estimator=Pipeline(memory=None,
                                         steps=[('ordinalencoder',
                                                 OrdinalEncoder(cols=[],
                                                                drop_invariant=False,
                                                                handle_missing='value',
                                                                handle_unknown='value',
                                                                mapping=[],
                                                                return_df=True,
                                                                verbose=0)),
                                                ('simpleimputer',
                                                 SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                    

In [139]:
X = X_train[['Player_A', 'Player_B', 'Player A bpFaced', 
             'Player B bpFaced','Player A bpSaved', 'Player B bpSaved']]

pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='median'),
    XGBClassifier(n_estimators=100, n_jobs=-1)
)

pipeline.fit(X, y_train)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['Player_A', 'Player_B'],
                                drop_invariant=False, handle_missing='value',
                                handle_unknown='value',
                                mapping=[{'col': 'Player_A',
                                          'data_type': dtype('O'),
                                          'mapping': Craig Wittus                  1
David Engel                   2
Bob Hewitt                    3
Felix Mantilla                4
Todd Woodbridge               5
                           ... 
Billy Knight               4472
Ponkaphan Phisaimongket    4473
Emmanuel Van Der Pol       4474
Wan Gao...
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, m

In [142]:
def predict(Player_A, Player_B, a, b, c, d):
    df = pd.DataFrame(
        columns = ['Player_A', 'Player_B', 'Player A bpFaced', 
             'Player B bpFaced','Player A bpSaved', 'Player B bpSaved'],
        data=[[Player_A, Player_B, a, b, c, d]]
    )
    y_pred = pipeline.predict(df)[0]
    return y_pred

In [146]:
predict('Novak Djokovic', 'Andy Roddick', 10, 10, 100, 30)

'Player A'

In [29]:
feature_names = X_val.columns.tolist()
eli5.show_weights(
    permuter, 
    top=None, # show permutation importances for all features
    feature_names=feature_names
)

Weight,Feature
0.1303 ± 0.0035,Player A bpFaced
0.1191 ± 0.0010,Player B bpFaced
0.0380 ± 0.0021,Player A bpSaved
0.0325 ± 0.0002,Player B 1stWon
0.0312 ± 0.0039,Player A rank
0.0305 ± 0.0039,Player A 1stWon
0.0286 ± 0.0011,Player B rank
0.0273 ± 0.0022,Player B bpSaved
0.0118 ± 0.0022,Player B
0.0104 ± 0.0000,Player A 2ndwon


In [18]:
eval_set = [(X_train_processed, y_train), 
             (X_val_processed, y_val)]

model.fit(X_train_processed, y_train, eval_set=eval_set, eval_metric='auc', 
          early_stopping_rounds=50
         )     #ROC AUC score.

[0]	validation_0-auc:0.728141	validation_1-auc:0.730131
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 50 rounds.
[1]	validation_0-auc:0.772831	validation_1-auc:0.772349
[2]	validation_0-auc:0.775189	validation_1-auc:0.776627
[3]	validation_0-auc:0.778009	validation_1-auc:0.778398
[4]	validation_0-auc:0.781137	validation_1-auc:0.781997
[5]	validation_0-auc:0.782912	validation_1-auc:0.78295
[6]	validation_0-auc:0.785422	validation_1-auc:0.785653
[7]	validation_0-auc:0.791062	validation_1-auc:0.792871
[8]	validation_0-auc:0.801283	validation_1-auc:0.799253
[9]	validation_0-auc:0.803381	validation_1-auc:0.801127
[10]	validation_0-auc:0.810535	validation_1-auc:0.807675
[11]	validation_0-auc:0.811976	validation_1-auc:0.809229
[12]	validation_0-auc:0.812302	validation_1-auc:0.809921
[13]	validation_0-auc:0.817985	validation_1-auc:0.814594
[14]	validation_0-auc:0.81881	validation_1-auc:0.815259


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=-1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [68]:
from sklearn.metrics import roc_auc_score
X_test_processed = processor.transform(X_test)
class_index = 1
y_pred_proba = model.predict_proba(X_test_processed)[:, class_index]    
print(f'Test ROC AUC for class {class_index}:')
print(roc_auc_score(y_test, y_pred_proba)) 

Test ROC AUC for class 1:
0.8781000689891166


In [66]:
RF = X_train[(X_train['Player B'].str.contains('Roger Federer'))
                      & (X_train['Player A'].str.contains('Rafael Nadal'))]
RF1 = X_train[(X_train['Player A'].str.contains('Roger Federer'))
                       & (X_train['Player B'].str.contains('Rafael Nadal'))]
RvR = pd.concat([RF, RF1])

RF.shape, RF1.shape, RvR.shape

((11, 46), (16, 46), (27, 46))

In [67]:
row = RvR.iloc[[0]]
row

,Player A,Player A 1stIn,Player A 1stWon,Player A 2ndwon,Player A Entry,Player A Hand,Player A Height,Player A ID,Player A Name,Player A SvGms,Player A ace,Player A bpFaced,Player A bpSaved,Player A df,Player A ioc,Player A points,Player A rank,Player A seed,Player A svpt,Player B,Player B 1stIn,Player B 1stWon,Player B 2ndwon,Player B Entry,Player B Hand,Player B Height,Player B ID,Player B Name,Player B SvGms,Player B ace,Player B bpFaced,Player B bpSaved,Player B df,Player B ioc,Player B points,Player B rank,Player B seed,Player B svpt,best_of,draw_size,match_num,round,surface,tourney_id,tourney_level,year
71995,Rafael Nadal,35.0,27.0,9.0,NaN,L,185.0,104745,26.76,9.0,1.0,2.0,1.0,2.0,ESP,6105.0,5.0,5.0,52.0,Roger Federer,48.0,30.0,8.0,NaN,R,185.0,103819,31.58,9.0,4.0,11.0,7.0,0.0,SUI,9535.0,2.0,2.0,75.0,3,NaN,92,QF,Hard,2013-404,M,2013


In [64]:
import shap

explainer = shap.TreeExplainer(model)
row_processed = processor.transform(row)
shap_values = explainer.shap_values(row_processed)

shap.initjs()
shap.force_plot(
    base_value=explainer.expected_value, 
    shap_values=shap_values, 
    features=row
) 

Exception: Model type not yet supported by TreeExplainer: <class 'sklearn.pipeline.Pipeline'>

In [22]:
y_test.iloc[[102]]

73314    Player A
Name: Winner2, dtype: object

In [56]:
from pdpbox.pdp import pdp_isolate, pdp_plot

feature= 'Player A bpFaced'

isolated = pdp_isolate(
    model=model,
    dataset=X_val,
    model_features=X_val.columns,
    feature=feature
)

pdp_plot(isolated, feature_name=feature);

ValueError: No objects to concatenate